In [2]:
!pip3 install tiktoken


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [21]:
import tiktoken

# r50k_base vocab size: 50,257 https://arxiv.org/pdf/2404.09894
enc = tiktoken.get_encoding("r50k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

def tokenize(string):
    return enc.encode(string)

In [33]:
enc.n_vocab

50257

In [19]:
import datasets

dataset = datasets.load_dataset('karpathy/tiny_shakespeare')['train']

In [28]:
dataset_tok = tokenize(dataset["text"][0])

In [30]:
dataset_tok

Type:        list
String form: [5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 1 <...> 97, 5603, 25, 198, 16973, 1683, 22427, 4145, 308, 28130, 355, 314, 30, 198, 1537, 508, 2058, 994]
Length:      301966
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.

# Try an MLP
initial network code from mistral

In [68]:
import torch
import torch.nn as nn
import torch.optim as optim

class MLP(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.embedding(x)
        print(out)
        out = out.mean(dim=1)  # Average the embeddings to get a single vector per sentence
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [69]:
model = MLP(vocab_size=50_272, embedding_dim=2048, hidden_size=512, num_classes=50_272)

In [37]:
model

MLP(
  (embedding): Embedding(50272, 2048)
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=512, out_features=50272, bias=True)
)

In [44]:
p=next(model.parameters())
p

Parameter containing:
tensor([[-0.1237,  0.6950,  0.9105,  ..., -2.3125, -0.0577, -0.8628],
        [-1.0967, -1.8143, -0.5356,  ...,  0.5066,  1.9037,  0.7108],
        [ 0.3591,  1.1603,  0.0509,  ..., -0.6011,  0.0869, -0.7442],
        ...,
        [ 2.0267,  0.1178, -0.6515,  ..., -2.4884, -1.0660,  1.9106],
        [ 1.5039, -0.5427,  1.9437,  ..., -0.2915, -1.3808,  1.1161],
        [ 0.5985,  0.4771, -0.1294,  ...,  1.3846, -2.0148,  1.5337]],
       requires_grad=True)

In [47]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

129795680

In [103]:
import tiktoken

# r50k_base vocab size: 50,257 https://arxiv.org/pdf/2404.09894
enc = tiktoken.get_encoding("r50k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

def tokenize(string):
    return torch.tensor(enc.encode(string))

def detokenize(tensor):
    return enc.decode([tensor.item()])

In [56]:
input_tok=torch.tensor(tokenize("hello world"))

/var/folders/fd/9xbk36z13vx8c3h6tbf6qchm0000gn/T/ipykernel_73707/3084114126.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tok=torch.tensor(tokenize("hello world"))


In [59]:
model.embedding(input_tok)

tensor([[ 0.8269, -0.2897, -0.8429,  ..., -2.4529, -1.4156, -0.5684],
        [-0.5045,  0.1804, -0.7209,  ...,  0.0440,  0.2390, -0.6576]],
       grad_fn=<EmbeddingBackward0>)

In [62]:
model.embedding(input_tok).size()

torch.Size([2, 2048])

In [64]:
model.fc1(model.embedding(input_tok))

tensor([[ 0.0716, -0.4247,  0.0595,  ...,  0.7414, -0.4766,  0.0736],
        [ 0.6885, -0.0650, -0.0086,  ...,  0.3499, -0.9063, -0.0974]],
       grad_fn=<AddmmBackward0>)

In [71]:
model.fc1(model.embedding(input_tok)).size()

torch.Size([2, 512])

In [75]:
model.fc1(model.embedding(input_tok)).mean(dim=0).size()

torch.Size([512])

In [87]:
import torch
import torch.nn as nn
import torch.optim as optim

class MLP(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.embedding(x)
        out = out.mean(dim=0)  # Average the embeddings to get a single vector per sentence
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out
model = MLP(vocab_size=50_272, embedding_dim=2048, hidden_size=512, num_classes=50_272)

In [88]:
model(input_tok)

tensor([-0.1296, -0.0484,  0.2392,  ...,  0.0882, -0.2404,  0.1858],
       grad_fn=<ViewBackward0>)

In [90]:
model(input_tok).size()

torch.Size([50272])

In [91]:
output = model(input_tok)

In [92]:
torch.argmax(output)

tensor(15694)

In [100]:
torch.argmax(output).item()

15694

In [102]:
enc.decode([torch.argmax(output).item()])

' Oscar'

In [106]:
detokenize(torch.argmax(output))

' Oscar'

## training loop

In [133]:
def tokenize_dataset():
    import datasets
    dataset = datasets.load_dataset('karpathy/tiny_shakespeare')['train']
    return tokenize(dataset["text"][0])
dataset_tok = tokenize_dataset().to("mps")
dataset_tok_samples = torch.split(dataset_tok, 100)

In [146]:
dataset_tok_samples = torch.split(dataset_tok, 100)

In [150]:
len(dataset_tok_samples)

3020

In [127]:
i=0
x = dataset_tok_samples[i]
display(x)
y = dataset_tok_samples[i+1][0]
display(y)

tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11])

tensor(3285)

In [128]:
dataset_tok_samples[0]

tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11])

In [129]:
dataset_tok_samples[1]

tensor([3285,  502, 2740,   13,  198,  198, 3237,   25,  198, 5248])

In [121]:
import torch
import torch.nn as nn
import torch.optim as optim

class MLP(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.embedding(x)
        out = out.mean(dim=0)  # Average the embeddings to get a single vector per sentence
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out
        
model = MLP(vocab_size=50_272, embedding_dim=1024, hidden_size=512, num_classes=50_272)
model.to("mps")

MLP(
  (embedding): Embedding(50272, 1024)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=512, out_features=50272, bias=True)
)

In [120]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

77792864

In [155]:
from tqdm import tqdm 

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    print(f"epoch {epoch}")
    for i in tqdm(range(len(dataset_tok_samples)-1)):
        # Forward pass
        inputs = dataset_tok_samples[i]
        labels = dataset_tok_samples[i+1][0]

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        if (i % 100 == 0):
            print(loss.item())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

epoch 0


  1%|▎                                                 | 19/3019 [00:00<00:16, 185.00it/s]

5.465605735778809


  4%|██                                                | 121/3019 [00:02<00:53, 54.34it/s]

6.946822643280029


...

7.723869323730469


 84%|████████████████████████████████████████▉        | 2521/3019 [00:54<00:09, 51.70it/s]

7.844557762145996


 84%|█████████████████████████████████████████▏       | 2535/3019 [00:54<00:10, 46.18it/s]


KeyboardInterrupt: 

In [139]:
criterion = nn.CrossEntropyLoss()
i=0
inputs = dataset_tok_samples[i]
labels = dataset_tok_samples[i+1][0]
outputs = model(inputs)


In [140]:
loss = criterion(outputs, labels)

In [141]:
loss

tensor(10.7479, device='mps:0', grad_fn=<NllLossBackward0>)

In [156]:
model(tokenize("hello world"))

RuntimeError: Placeholder storage has not been allocated on MPS device!

In [159]:
import tiktoken

# r50k_base vocab size: 50,257 https://arxiv.org/pdf/2404.09894
enc = tiktoken.get_encoding("r50k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

def tokenize(string):
    return torch.tensor(enc.encode(string)).to("mps")

def detokenize(tensor):
    return enc.decode([tensor.item()])

In [161]:
torch.argmax(model(tokenize("hello world")))

tensor(616, device='mps:0')

In [170]:
def generate(prompt, tokens):
    with torch.no_grad():
        for _ in range(tokens):
            input_tok = tokenize(prompt)
            output_tok = model(input_tok)
            output = detokenize(torch.argmax(output_tok))
            prompt = prompt + output
        return prompt

In [172]:
generate("my", 100)

'my.\n:\n lord lord:\n lord lord,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,:,'

In [175]:
generate("First Citizen:\nBefore we proceed any", 50)

'First Citizen:\nBefore we proceed any lord lord be be be be.\n:\n:\n be be be be be a a a a a a,,,,,,,,,,,,,,,,,,,,,,,,,,,'

## training loop with generate

In [179]:
from tqdm import tqdm 

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    print(f"epoch {epoch}")
    for i in tqdm(range(len(dataset_tok_samples)-1)):
        # Forward pass
        inputs = dataset_tok_samples[i]
        labels = dataset_tok_samples[i+1][0]

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        if (i % 100 == 0):
            print(loss.item())
        if (i % 200 == 0):
            print(generate("Oh my lord", 50))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

epoch 0


  0%|                                                            | 0/3019 [00:00<?, ?it/s]

4.739320755004883
Oh my lord,,,,,,,,:
,,,,,,,,,,,,,,,,,,,,,,,,, you you you you you you you you you you you you you you you


  4%|██                                                | 121/3019 [00:02<00:52, 54.95it/s]

5.493747711181641


...

 97%|███████████████████████████████████████████████▍ | 2921/3019 [01:03<00:01, 55.01it/s]

4.103275299072266


100%|█████████████████████████████████████████████████| 3019/3019 [01:05<00:00, 46.20it/s]

4.535262584686279
Oh my lord: not not not to to have have to have have have have have have have have have have to have have haveI haveI haveI haveI to have have haveI have haveI haveI haveI haveI haveI have haveI have


Epoch [10/10], Loss: 3.7904


## training loop with generate and stats

In [1]:
import tiktoken

# r50k_base vocab size: 50,257 https://arxiv.org/pdf/2404.09894
enc = tiktoken.get_encoding("r50k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

def encode(string):
    return torch.tensor(enc.encode(string))

def decode(tensor):
    return enc.decode([tensor.item()])

In [8]:
import torch 

def tokenize_dataset():
    import datasets
    dataset = datasets.load_dataset('karpathy/tiny_shakespeare')['train']
    return encode(dataset["text"][0])

def get_dataloader(dataset_tok, sample_length):
    for i in range(len(dataset_tok)-sample_length):
        yield dataset_tok[i:i+sample_length], dataset_tok[i+sample_length]

dataset_tok = tokenize_dataset().to("mps")
dataset_samples = get_dataloader(dataset_tok, 10)


In [12]:
len(dataset_tok)

301966

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

class MLP(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.embedding(x)
        out = out.mean(dim=0)  # Average the embeddings to get a single vector per sentence
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def generate(prompt, tokens):
    with torch.no_grad():
        for _ in range(tokens):
            input_tok = encode(prompt).to("mps")
            output_tok = model(input_tok)
            output = decode(torch.argmax(output_tok))
            prompt = prompt + output
        return prompt
        
model = MLP(vocab_size=50_272, embedding_dim=1024, hidden_size=512, num_classes=50_272)
model.to("mps")

count_parameters(model)

77792864

In [ ]:
from tqdm import tqdm 
import time

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
curr_time = time.time()
curr_step = 1
curr_tok = 0

for epoch in range(num_epochs):
    print(f"epoch {epoch}")
    for inputs, labels in dataset_samples:
        tok_cnt = inputs.size()[0]
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (curr_step % 20 == 0):
            step_time = time.time() - curr_time
            print(f"step {curr_step}: loss {loss.detach().item():.2f} {step_time*1000:.2f}ms {(tok_cnt/step_time):.0f}tok/s (total {curr_tok:,} tok)")

        if (curr_step % 100 == 0):
            print(generate("Oh my lord", 10))
        
        curr_time = time.time() 
        curr_step = curr_step + 1
        curr_tok = curr_tok + tok_cnt

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

epoch 0
step 20: loss 8.78 20.90ms 479tok/s (total 190tok)
step 40: loss 11.80 21.63ms 462tok/s (total 390tok)
step 60: loss 10.90 21.33ms 469tok/s (total 590tok)
step 80: loss 7.33 20.71ms 483tok/s (total 790tok)
step 100: loss 11.63 21.12ms 473tok/s (total 990tok)
Oh my lord
:,:,.
,.,
step 120: loss 11.16 20.75ms 482tok/s (total 1,190tok)
step 140: loss 5.41 20.67ms 484tok/s (total 1,390tok)
step 160: loss 11.04 20.72ms 483tok/s (total 1,590tok)
step 180: loss 6.27 20.70ms 483tok/s (total 1,790tok)
step 200: loss 12.16 20.68ms 484tok/s (total 1,990tok)
Oh my lord,
 a a a, a
 a was
step 220: loss 7.09 21.03ms 475tok/s (total 2,190tok)
step 240: loss 13.34 20.67ms 484tok/s (total 2,390tok)
step 260: loss 11.08 20.77ms 481tok/s (total 2,590tok)
step 280: loss 5.48 21.66ms 462tok/s (total 2,790tok)
step 300: loss 11.73 20.79ms 481tok/s (total 2,990tok)
Oh my lord
:,:
, he he the in
step 320: loss 8.03 20.96ms 477tok/s (total 3,190tok)
step 340: loss 11.07 20.71ms 483tok/s (total 3,390tok

In [13]:
dataset_tok_samples[0].size()[0]

100